# Import Basic Libraries

In [1]:
import pandas as pd
import numpy as np

# Import libraries for ml methods
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer, matthews_corrcoef, balanced_accuracy_score, \
    f1_score, fbeta_score, recall_score, precision_score, average_precision_score, accuracy_score

# Data loading

In [2]:
data = pd.read_csv('Hepatitis_C.csv')
data

,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,label
0,32,0,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7,0
1,45,0,41.7,73.2,43.6,29.4,6.4,8.89,5.31,71.0,67.4,70.3,0
2,55,0,41.5,59.5,15.4,16.2,6.8,6.35,5.22,80.0,12.4,69.9,0
3,53,0,37.8,98.1,30.5,21.1,4.0,5.02,4.42,94.0,23.2,65.2,0
4,56,1,39.7,66.0,14.2,20.8,3.5,7.48,5.88,66.0,7.2,67.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,62,1,32.0,416.6,5.9,110.3,50.0,5.57,6.30,55.7,650.9,68.5,1
200,64,1,24.0,102.8,2.9,44.4,20.0,1.54,3.02,63.0,35.9,71.3,1
201,64,1,29.0,87.3,3.5,99.0,48.0,1.66,3.63,66.7,64.2,82.0,1
202,46,1,33.0,62.7,39.0,62.0,20.0,3.56,4.20,52.0,50.0,71.0,1


## Split data to X and y

In [3]:
X = data.drop('label', axis=1).copy()
y = data['label'].copy()

print(X.shape, y.shape)

(204, 12) (204,)


# Normalize data

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = scaler.fit_transform(X)

print(X.shape, y.shape)

(204, 12) (204,)


## Train / Test split of data

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, \
                                                    stratify=y, random_state=42)

### Define the ML models

In [6]:
# Define classifiers
classifiers = {
    'LR':  LogisticRegression(),
    'GNB': GaussianNB(),
    'kNN': KNeighborsClassifier(),
    'LDA': LinearDiscriminantAnalysis(),
    'SVM': SVC()
}

# Build nested Cross Validation (nCV) pipeline

For the outer loop we will use K=5 folds and for the inner loop L=3 folds.

In [7]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

cv_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [8]:
# Inner cross-validation for parameter search
param_grid = {"C": [0.1, 1, 10], "gamma": [.01, .1]}
clf = SVC()

model = GridSearchCV(
    estimator=clf, param_grid=param_grid, 
    scoring='f1_macro', cv=cv_inner, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, X_train, y_train, scoring='matthews_corrcoef', 
                             cv=cv_outer, n_jobs=-1)

print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.846 ± 0.146


In [9]:
test_score_nested = []

N_TRIALS = 10
for i in range(N_TRIALS):
    # For each trial, we use cross-validation splits on independently
    # randomly shuffled data by passing distinct values to the random_state
    # parameter.

    # Non_nested parameter search and scoring
    model = GridSearchCV(estimator=clf, param_grid=param_grid,
                         cv=cv_inner, n_jobs=2)
    
    model.fit(X_train, y_train)

    # Nested CV with parameter optimization
    test_score = cross_val_score(model, X_train, y_train, cv=cv_outer, n_jobs=2)

    test_score_nested.append(test_score.mean())

In [10]:
all_scores = pd.DataFrame(test_score_nested)

In [11]:
all_scores

,0
0,0.932008
1,0.932008
2,0.932008
3,0.932008
4,0.932008
5,0.932008
6,0.932008
7,0.932008
8,0.932008
9,0.932008
